<a href="https://colab.research.google.com/github/mrmarken/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Install Postgres Driver
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://develo

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.5.0.jar").getOrCreate()


In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...|2015-08-31 00:00:00|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with

In [4]:
# Clean the DataFrame
cleaned_df = df.dropna()
cleaned_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...|2015-08-31 00:00:00|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with

In [5]:
# Create the vine_table DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RY01SAV7HZ8QO|          4|            0|          0|   N|                Y|
|R1XX8SDGJ4MZ4L|          5|            0|          0|   N|                N|
|R149Q3B5L33NN5|          5|            0|          0|   N|                Y|
|R2ZVD69Z6KPJ4O|          1|            0|          2|   N|                Y|
|R1DIKG2G33ZLNP|          4|            0|          0|   N|                Y|
|R3L6FGKAW0EYFI|          1|            1|          1|   N|                N|
| RAO0QZH5VC6VI|          1|            0|          0|   N|                Y|
|R25IK0UAHWNB22|          3|            0|          0|   N|                Y|
|R2A9IHKZMTMAL1|          5|            1|          2|   N|                Y|
| R5XVKTHL6SITI|          5|            0|          0|   N|     

# Deliverable 2: Determine Bias of Vine Reviews

In [6]:
# Step 1: Filter the data and create a new DataFrame or table to retrieve all the 
# rows where the total_votes count is equal to or greater than 20

from pyspark.sql.functions import col

total_votes_df = vine_df.filter(col('total_votes') >= 20)
total_votes_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|
|R1LV778IDGRJQ6|          1|            4|         23|   N|                Y|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|
|R2LBEAGVJA7HK1|          3|            5|         48|   N|                N|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|
|R3QLX5NODD1OJ4|          1|            9|         25|   N|                Y|
|R2SOOHE0XUQON1|          1|            2|         24|   N|     

In [7]:
# Step 2: Filter the new DataFrame or table created in Step 1 and create a new 
# DataFrame or table to retrieve all the rows where the number of helpful_votes 
# divided by total_votes is equal to or greater than 50%.

votes_above_50_df = total_votes_df.withColumn('votes_above_50',col('helpful_votes')/col('total_votes')).alias('votes_above_50').filter(col("votes_above_50") >= 0.5)
votes_above_50_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    votes_above_50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|             0.825|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|0.8936170212765957|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|               0.7|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|0.9226804123711341|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|0.8571428571428571|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|0.8333333333333334|
|R2I9AO0VEBYJES|          4|           47|         57|   N|                Y|0.8245614035087719|
|R3Q7YJX1I5PDMU|          3|  

In [8]:
# Step 3: Filter the DataFrame or table created in Step 2, and create a new 
# DataFrame or table that retrieves all the rows where a review was written as 
# part of the Vine program (paid), vine == 'Y'

paid_df = votes_above_50_df.filter(col('vine') == 'Y')
paid_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    votes_above_50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3LLICBKRHLRJC|          4|           17|         22|   Y|                N|0.7727272727272727|
| RUD135S06DEVE|          5|           41|         51|   Y|                N| 0.803921568627451|
|R2WG8UL4SGDPDH|          4|           23|         25|   Y|                N|              0.92|
|R158AOKOA28WS2|          4|           18|         28|   Y|                N|0.6428571428571429|
| RGWV3QLJQXU9X|          5|          332|        380|   Y|                N|0.8736842105263158|
| RD0E1RZSSYGGN|          5|          261|        292|   Y|                N|0.8938356164383562|
|R3LGCTQH5ZMGO0|          5|           61|         68|   Y|                N|0.8970588235294118|
|R2H24YTRLARCZJ|          4|  

In [9]:
# Step 4: Repeat Step 3, but this time retrieve all the rows where the review 
# was not part of the Vine program (unpaid), vine == 'N'.

unpaid_df = votes_above_50_df.filter(col('vine') == 'N')
unpaid_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    votes_above_50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|             0.825|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|0.8936170212765957|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|               0.7|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|0.9226804123711341|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|0.8571428571428571|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|0.8333333333333334|
|R2I9AO0VEBYJES|          4|           47|         57|   N|                Y|0.8245614035087719|
|R3Q7YJX1I5PDMU|          3|  

In [11]:
# Step 5: Determine the total number of reviews, the number of 5-star reviews, and 
# the percentage of 5-star reviews for the two types of review (paid vs unpaid).

from pyspark.sql.functions import col,when,count,lit
reviews_total_df = votes_above_50_df.groupBy("vine").agg(
    count(col('vine')).alias('Total_Reviews'),
    count(when(col('star_rating') == 5, True)).alias('Total_5_Star_Reviews'),
    (count(when(col('star_rating') == 5, True))/count(col('Vine'))*100).alias('Percent_5_Star_Total'))
reviews_total_df.show()


+----+-------------+--------------------+--------------------+
|vine|Total_Reviews|Total_5_Star_Reviews|Percent_5_Star_Total|
+----+-------------+--------------------+--------------------+
|   Y|          261|                 106|   40.61302681992337|
|   N|        24040|               10899|    45.3369384359401|
+----+-------------+--------------------+--------------------+

